In [1]:
import os
import time
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torchvision

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.models.detection.anchor_utils import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FasterRCNN, FastRCNNPredictor
from torchvision.transforms import ToTensor, Compose, RandomHorizontalFlip,\
    RandomVerticalFlip, Resize, Normalize
from sklearn.model_selection import train_test_split
from PIL import Image, ImageFile, ImageFilter

In [15]:
def rle2bbox(rle, shape):
    '''
    rle: run-length encoded image mask, as string
    shape: (height, width) of image on which RLE was produced
    Returns (x0, y0, x1, y1) tuple describing the bounding box of the rle mask
    
    Note on image vs np.array dimensions:
    
        np.array implies the `[y, x]` indexing order in terms of image dimensions,
        so the variable on `shape[0]` is `y`, and the variable on the `shape[1]` is `x`,
        hence the result would be correct (x0,y0,x1,y1) in terms of image dimensions
        for RLE-encoded indices of np.array (which are produced by widely used kernels
        and are used in most kaggle competitions datasets)
    '''
    
    a = np.fromiter(rle.split(), dtype=np.uint)
    a = a.reshape((-1, 2))  # an array of (start, length) pairs
    a[:,0] -= 1  # `start` is 1-indexed
    
    y0 = a[:,0] % shape[0]
    y1 = y0 + a[:,1]
    if np.any(y1 > shape[0]):
        # got `y` overrun, meaning that there are a pixels in mask on 0 and shape[0] position
        y0 = 0
        y1 = shape[0]
    else:
        y0 = np.min(y0)
        y1 = np.max(y1)
    
    x0 = a[:,0] // shape[0]
    x1 = (a[:,0] + a[:,1]) // shape[0]
    x0 = np.min(x0)
    x1 = np.max(x1)
    
    if x1 > shape[1]:
        # just went out of the image dimensions
        raise ValueError("invalid RLE or image dimensions: x1=%d > shape[1]=%d" % (
            x1, shape[1]
        ))

    return x0, y0, x1, y1

In [3]:
def make_model(state_dict, num_classes):
        inception = torchvision.models.inception_v3(pretrained=False, progress=False, 
                                                    num_classes=num_classes, aux_logits=False)
        inception.load_state_dict(torch.load(state_dict))
        modules = list(inception.children())[:-1]
        backbone = nn.Sequential(*modules)

        for layer in backbone:
            for p in layer.parameters():
                p.requires_grad = False # Freezes the backbone layers

        backbone.out_channels = 2048

        anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                           aspect_ratios=((0.5, 1.0, 2.0),))

        model = FasterRCNN(backbone, rpn_anchor_generator=anchor_generator,
                           box_predictor=FastRCNNPredictor(1024, num_classes))

        return model

In [13]:
def is_valid(rle, shape=(768,768)):
    width, height = shape
    xmin, ymin, xmax, ymax = rle2bbox(rle, shape)
    if xmin >= 0 and xmax <= width and xmin < xmax and \
    ymin >= 0 and ymax <= height and ymin < ymax:
        return True
    return False

In [20]:
ship_dir = '../dev/'
train_image_dir = os.path.join(ship_dir, 'imgs/')
valid_image_dir = os.path.join(ship_dir, 'imgs/')
masks = pd.read_csv(os.path.join(ship_dir,
                                 'train_ship_segmentations_v2.csv'))

In [27]:
grp = list(masks.groupby('ImageId'))
image_ids =  [_id for _id, _ in grp] 
image_masks = [m['EncodedPixels'].values for _,m in grp]

In [31]:
def make_target(in_mask_list, N, shape=(768, 768)):
    if N == 0:
        target = {}
        target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
        target["labels"] = torch.zeros((0), dtype=torch.int64)
        return target
    bbox_array = np.empty((N, 4), dtype=np.float32)
    labels = torch.ones((N,), dtype=torch.int64)
    i = 0
    for rle in in_mask_list:
        if isinstance(rle, str):
            # bbox = tuple(x1, y1, x2, y2)
            bbox = rle2bbox(rle, shape)
            bbox_array[i,:] = bbox
        i += 1
    target = {
        'boxes': torch.from_numpy(bbox_array),
        'labels': labels,
    }
    return target

In [38]:
in_mask_list = image_masks[3]
N = sum([1 for i in in_mask_list if isinstance(i, str)])
make_target(in_mask_list, N, shape=(768,768))

{'boxes': tensor([[469., 287., 491., 307.],
         [ 67., 377.,  84., 386.],
         [258., 174., 305., 185.],
         [ 72., 386.,  78., 387.],
         [331., 178., 369., 197.]]),
 'labels': tensor([1, 1, 1, 1, 1])}

In [17]:
ship_dir = '../dev/'
train_image_dir = os.path.join(ship_dir, 'imgs/')
valid_image_dir = os.path.join(ship_dir, 'imgs/')
masks = pd.read_csv(os.path.join(ship_dir,
                                 'train_ship_segmentations_v2.csv'))
unique_img_ids = masks.groupby('ImageId').reset_index(name='counts')
train_ids, valid_ids = train_test_split(unique_img_ids, 
                 test_size = 0.01, 
                 stratify = unique_img_ids['counts'],
                 random_state=seed
                )

In [19]:
labeled_masks.head()

,ImageId,counts
0,00003e153.jpg,True
1,0001124c7.jpg,True
2,000155de5.jpg,True
3,000194a2d.jpg,True
4,0001b1832.jpg,True


In [ ]:
train_ids, valid_ids = train_test_split(unique_img_ids, 
                 test_size = 0.01, 
                 stratify = unique_img_ids['counts'],
                 random_state=seed
                )
print("Train Size: %d" % len(train_ids))
print("Valid Size: %d" % len(valid_ids))
train_df = pd.merge(unique_img_ids, train_ids)
valid_df = pd.merge(unique_img_ids, valid_ids)

# Components

In [133]:
import pathlib
from typing import Callable, Iterator, Union, Optional, List, Tuple, Dict


def get_masks(ship_dir: str, 
                train_image_dir: Union[str, pathlib.Path], 
                valid_image_dir: Union[str, pathlib.Path]
               ) -> pd.DataFrame:
    masks = pd.read_csv(os.path.join(ship_dir,
                                     'train_ship_segmentations_v2.csv'
                                    )
                       )
    return masks


def is_valid(rle, shape=(768,768)) -> bool:
    width, height = shape
    xmin, ymin, xmax, ymax = rle2bbox(rle, shape)
    if xmin >= 0 and xmax <= width and xmin < xmax and \
    ymin >= 0 and ymax <= height and ymin < ymax:
        return True
    return False


def filter_masks(masks: pd.DataFrame) -> Tuple[dict, dict]:
    grp = list(masks.groupby('ImageId'))
    image_names =  {idx: filename for idx, (filename, _) in enumerate(grp)} 
    image_masks = {idx: m['EncodedPixels'].values for idx, (_, m) in enumerate(grp)}
    to_remove = []
    for idx, in_mask_list in image_masks.items():
        N = sum([1 for i in in_mask_list if isinstance(i, str)])
        if N > 0:
            for i, rle in enumerate(in_mask_list):
                if not is_valid(rle):
                    to_remove.append(idx)
                    
    for idx in to_remove:
        del image_names[idx]
        del image_masks[idx]
    return image_names, image_masks
        

def get_train_valid_dfs(masks: dict, seed: int = 0) -> Tuple[list, list, list, list]:
    ids = np.array(list(masks.keys())).reshape((len(masks),1))
    train_ids, valid_ids = train_test_split(
         ids, 
         test_size = 0.01, 
         random_state=seed
        )
    train_ids, valid_ids = list(train_ids.flatten()), list(valid_ids.flatten())
    train_masks = [masks[idx] for idx in train_ids]
    valid_masks = [masks[idx] for idx in valid_ids]
    return train_ids, train_masks, valid_ids, valid_masks


if __name__ == '__main__':
    ship_dir = '../dev/'
    train_image_dir = os.path.join(ship_dir, 'imgs/')
    valid_image_dir = os.path.join(ship_dir, 'imgs/')
    masks = get_masks(ship_dir, train_image_dir, valid_image_dir)
    image_names, filtered_masks = filter_masks(masks)
    train_ids, train_masks, valid_ids, valid_masks = get_train_valid_dfs(
        filtered_masks)

[179451, 92763, 165924, 179608, 127945]
[nan]


In [178]:
len(train_masks)

190620

In [157]:
in_mask_list = train_masks[0]
N = sum([1 for i in in_mask_list if isinstance(i, str)])

make_target(in_mask_list, N, shape=(768, 768))['boxes'].shape

torch.Size([2, 4])

In [132]:
ids = np.array([[1], [2]])
print(list(ids.flatten()))
masks = np.array([69, 420, 1189998119991197253])
[masks[idx] for idx in ids]

[1, 2]


[array([420]), array([1189998119991197253])]

In [98]:
from torchvision.transforms.functional import resize


class Resize:
    def __init__(self, 
                 input_shape = (768, 768), 
                 output_shape = (299, 299), 
                 interpolation=2
                ):
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.interpolation = interpolation
        
        
    def resize_boxes(self, boxes: torch.tensor) -> torch.tensor:
        x_orig, y_orig = self.input_shape
        x_new, y_new = self.output_shape
        x_scale = x_new / x_orig
        y_scale = y_new / y_orig
        # bbox = tuple(x1, y1, x2, y2)
        row_scaler = torch.tensor([x_scale, y_scale, x_scale, y_scale])
        boxes_scaled = torch.round(boxes * row_scaler).int() # Converts to new coordinates
        return boxes_scaled
        
        
    def __call__(self, image, target) -> Tuple[torch.tensor, dict]:
        image = resize(image, size=self.output_shape, interpolation=self.interpolation)
        target['boxes'] = self.resize_boxes(target['boxes'])
        return image, target

In [159]:
def test_resize():
    rgb_path = r'../dev/imgs/0002756f7.jpg'
    target = {}
    target['boxes'] = torch.tensor([[100,100, 200, 200],[10,11,12,13],[14,15,16,17]])
    image =  Image.open(rgb_path)
    Resize()(image, target)
    print("test passed")

test_resize()

test passed


# Test `VesselDataset`

In [166]:
class RandomBlur:
    def __init__(self, p=0.5, radius=2):
        self.p = p
        self.radius = radius

        
    def __call__(self, x):
        prob = np.random.rand(1)[0]
        if prob < self.p:
            x = x.filter(ImageFilter.GaussianBlur(self.radius))
        return x

In [188]:
class VesselDataset(Dataset):
    def __init__(self, boxes: Optional[list], image_names: list, train_image_dir=None, valid_image_dir=None, 
                 test_image_dir=None, transform=None, mode='train', binary=True):
        self.boxes = boxes
        self.image_names = image_names
        self.train_image_dir = train_image_dir
        self.valid_image_dir = valid_image_dir
        self.test_image_dir = test_image_dir

        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        if transform is not None:
            self.train_transform = transform
        else:
            self.train_transform = Compose([
                RandomBlur(p=0.95, radius=2),
                ToTensor(),
                Normalize(mean, std) # Apply to all input images
            ])
        self.valid_transform = Compose([
            RandomBlur(p=1.0, radius=2), # Blur all images
            ToTensor(),
            Normalize(mean, std) # Apply to all input images
        ])
        self.test_transform = Compose([
            transforms.Resize(size=(299,299), interpolation=2),
            ToTensor(),
            Normalize(mean, std) # Apply to all input images
        ])
        self.mode = mode


    def __len__(self):
        return len(self.boxes)


    def __getitem__(self, idx):
        img_file_name = self.image_names[idx]
        if self.mode == 'train':
            img_path = os.path.join(self.train_image_dir, img_file_name)
        elif self.mode == 'valid':
            img_path = os.path.join(self.valid_image_dir, img_file_name)
        else:
            img_path = os.path.join(self.test_image_dir, img_file_name)

        #img = imread(img_path)
        img = Image.open(img_path)
        if self.mode =='train' or self.mode =='valid':
            img_boxes = self.boxes[idx]
            N = sum([1 for i in img_boxes if isinstance(i, str)])
            target = make_target(img_boxes, N, shape=(768, 768))
            img, target = Resize(input_shape = (768, 768), 
                                 output_shape = (299, 299)
                                )(img, target)
        
        if self.mode =='train':
            img = self.train_transform(img)
            return img, target
        elif self.mode == 'valid':
            img = self.valid_transform(img)
            return img, target
        else:
            img = self.test_transform(img)
            return img

In [176]:

for ids, name in image_names.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
    if name == r'000d26c17.jpg':
        print(ids)

32


In [187]:
VesselDataset(train_masks, image_names, test_image_dir = r'../dev/imgs', mode='test').__getitem__(32).shape

000d26c17.jpg


torch.Size([3, 299, 299])

# Everything Together

In [197]:
import pathlib
from typing import Callable, Iterator, Union, Optional, List, Tuple, Dict
from torchvision.transforms.functional import resize


def rle2bbox(rle, shape):
    '''
    rle: run-length encoded image mask, as string
    shape: (height, width) of image on which RLE was produced
    Returns (x0, y0, x1, y1) tuple describing the bounding box of the rle mask
    
    Note on image vs np.array dimensions:
    
        np.array implies the `[y, x]` indexing order in terms of image dimensions,
        so the variable on `shape[0]` is `y`, and the variable on the `shape[1]` is `x`,
        hence the result would be correct (x0,y0,x1,y1) in terms of image dimensions
        for RLE-encoded indices of np.array (which are produced by widely used kernels
        and are used in most kaggle competitions datasets)
    '''
    
    a = np.fromiter(rle.split(), dtype=np.uint)
    a = a.reshape((-1, 2))  # an array of (start, length) pairs
    a[:,0] -= 1  # `start` is 1-indexed
    
    y0 = a[:,0] % shape[0]
    y1 = y0 + a[:,1]
    if np.any(y1 > shape[0]):
        # got `y` overrun, meaning that there are a pixels in mask on 0 and shape[0] position
        y0 = 0
        y1 = shape[0]
    else:
        y0 = np.min(y0)
        y1 = np.max(y1)
    
    x0 = a[:,0] // shape[0]
    x1 = (a[:,0] + a[:,1]) // shape[0]
    x0 = np.min(x0)
    x1 = np.max(x1)
    
    if x1 > shape[1]:
        # just went out of the image dimensions
        raise ValueError("invalid RLE or image dimensions: x1=%d > shape[1]=%d" % (
            x1, shape[1]
        ))

    return x0, y0, x1, y1


# From: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_decode(mask_rle, shape=(299, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction


def is_valid(rle, shape=(768,768)):
    width, height = shape
    xmin, ymin, xmax, ymax = rle2bbox(rle, shape)
    if xmin >= 0 and xmax <= width and xmin < xmax and \
    ymin >= 0 and ymax <= height and ymin < ymax:
        return True
    return False


def make_target(in_mask_list, N, shape=(768, 768)):
    if N == 0:
        target = {}
        target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
        target["labels"] = torch.zeros((0), dtype=torch.int64)
        return target
    bbox_array = np.empty((N, 4), dtype=np.float32)
    labels = torch.ones((N,), dtype=torch.int64)
    i = 0
    for rle in in_mask_list:
        if isinstance(rle, str):
            # bbox = tuple(x1, y1, x2, y2)
            bbox = rle2bbox(rle, shape)
            bbox_array[i,:] = bbox
        i += 1
    target = {
        'boxes': torch.from_numpy(bbox_array),
        'labels': labels,
    }
    return target


def get_masks(ship_dir: str, 
                train_image_dir: Union[str, pathlib.Path], 
                valid_image_dir: Union[str, pathlib.Path]
               ) -> pd.DataFrame:
    masks = pd.read_csv(os.path.join(ship_dir,
                                     'train_ship_segmentations_v2.csv'
                                    )
                       )
    return masks


def is_valid(rle, shape=(768,768)) -> bool:
    width, height = shape
    xmin, ymin, xmax, ymax = rle2bbox(rle, shape)
    if xmin >= 0 and xmax <= width and xmin < xmax and \
    ymin >= 0 and ymax <= height and ymin < ymax:
        return True
    return False


def filter_masks(masks: pd.DataFrame) -> Tuple[dict, dict]:
    grp = list(masks.groupby('ImageId'))
    image_names =  {idx: filename for idx, (filename, _) in enumerate(grp)} 
    image_masks = {idx: m['EncodedPixels'].values for idx, (_, m) in enumerate(grp)}
    to_remove = []
    for idx, in_mask_list in image_masks.items():
        N = sum([1 for i in in_mask_list if isinstance(i, str)])
        if N > 0:
            for i, rle in enumerate(in_mask_list):
                if not is_valid(rle):
                    to_remove.append(idx)
                    
    for idx in to_remove:
        del image_names[idx]
        del image_masks[idx]
    return image_names, image_masks
        

def get_train_valid_dfs(masks: dict, seed: int = 0) -> Tuple[list, list, list, list]:
    ids = np.array(list(masks.keys())).reshape((len(masks),1))
    train_ids, valid_ids = train_test_split(
         ids, 
         test_size = 0.01, 
         random_state=seed
        )
    train_ids, valid_ids = list(train_ids.flatten()), list(valid_ids.flatten())
    train_masks = [masks[idx] for idx in train_ids]
    valid_masks = [masks[idx] for idx in valid_ids]
    return train_ids, train_masks, valid_ids, valid_masks


class Resize:
    def __init__(self, 
                 input_shape = (768, 768), 
                 output_shape = (299, 299), 
                 interpolation=2
                ):
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.interpolation = interpolation
        
        
    def resize_boxes(self, boxes: torch.tensor) -> torch.tensor:
        x_orig, y_orig = self.input_shape
        x_new, y_new = self.output_shape
        x_scale = x_new / x_orig
        y_scale = y_new / y_orig
        # bbox = tuple(x1, y1, x2, y2)
        row_scaler = torch.tensor([x_scale, y_scale, x_scale, y_scale])
        boxes_scaled = torch.round(boxes * row_scaler).int() # Converts to new coordinates
        return boxes_scaled
        
        
    def __call__(self, image, target) -> Tuple[torch.tensor, dict]:
        image = resize(image, size=self.output_shape, interpolation=self.interpolation)
        target['boxes'] = self.resize_boxes(target['boxes'])
        return image, target
    
    
class RandomBlur:
    def __init__(self, p=0.5, radius=2):
        self.p = p
        self.radius = radius


    def __call__(self, x):
        prob = np.random.rand(1)[0]
        if prob < self.p:
            x = x.filter(ImageFilter.GaussianBlur(self.radius))
        return x
    
    
class VesselDataset(Dataset):
    def __init__(self, 
                 boxes: Optional[list], 
                 image_names: list, 
                 train_image_dir=None, 
                 valid_image_dir=None, 
                 test_image_dir=None, 
                 transform=None, 
                 mode='train', 
                 binary=True):
        self.boxes = boxes
        self.image_names = image_names
        self.train_image_dir = train_image_dir
        self.valid_image_dir = valid_image_dir
        self.test_image_dir = test_image_dir

        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        if transform is not None:
            self.train_transform = transform
        else:
            self.train_transform = Compose([
                RandomBlur(p=0.95, radius=2),
                ToTensor(),
                Normalize(mean, std) # Apply to all input images
            ])
        self.valid_transform = Compose([
            RandomBlur(p=1.0, radius=2), # Blur all images
            ToTensor(),
            Normalize(mean, std) # Apply to all input images
        ])
        self.test_transform = Compose([
            transforms.Resize(size=(299,299), interpolation=2),
            ToTensor(),
            Normalize(mean, std) # Apply to all input images
        ])
        self.mode = mode


    def __len__(self):
        return len(self.boxes)


    def __getitem__(self, idx):
        img_file_name = self.image_names[idx]
        if self.mode == 'train':
            img_path = os.path.join(self.train_image_dir, img_file_name)
        elif self.mode == 'valid':
            img_path = os.path.join(self.valid_image_dir, img_file_name)
        else:
            img_path = os.path.join(self.test_image_dir, img_file_name)

        #img = imread(img_path)
        img = Image.open(img_path)
        if self.mode =='train' or self.mode =='valid':
            img_boxes = self.boxes[idx]
            N = sum([1 for i in img_boxes if isinstance(i, str)])
            target = make_target(img_boxes, N, shape=(768, 768))
            img, target = Resize(input_shape = (768, 768), 
                                 output_shape = (299, 299)
                                )(img, target)
            # Make image_id
            image_id = torch.tensor([idx])
            target["image_id"] = image_id
        
        if self.mode =='train':
            img = self.train_transform(img)
            return img, target
        elif self.mode == 'valid':
            img = self.valid_transform(img)
            return img, target
        else:
            img = self.test_transform(img)
            return img

In [198]:
ship_dir = '../dev/'
train_image_dir = os.path.join(ship_dir, 'imgs/')
valid_image_dir = os.path.join(ship_dir, 'imgs/')
masks = get_masks(ship_dir, train_image_dir, valid_image_dir)
image_names, filtered_masks = filter_masks(masks)
train_ids, train_masks, valid_ids, valid_masks = get_train_valid_dfs(
    filtered_masks
)

vessel_dataset = VesselDataset(train_masks, image_names, train_image_dir = r'../dev/imgs', mode='train')

vessel_valid_dataset = VesselDataset(valid_masks, image_names, valid_image_dir = r'../dev/imgs', mode='valid')

batch_size = 64
shuffle = True
loader = DataLoader(
            dataset=vessel_dataset,
            shuffle=shuffle,
            #num_workers = 0,
            batch_size=batch_size,
            pin_memory=torch.cuda.is_available()
        )

valid_loader = DataLoader(
            dataset=vessel_valid_dataset,
            shuffle=shuffle,
            #num_workers = 0,
            batch_size=batch_size,
            pin_memory=torch.cuda.is_available()
        )

num_epochs = 30
print_freq = 100

KeyboardInterrupt: 

# Load Dummy Model and Test IO

In [193]:
# Adapted from https://discuss.pytorch.org/t/faster-rcnn-with-inceptionv3-backbone-very-slow/91455
def make_model(state_dict=None, num_classes=2):
        inception = torchvision.models.inception_v3(pretrained=False, progress=False, 
                                                    num_classes=num_classes, aux_logits=False)
        #inception.load_state_dict(torch.load(state_dict))
        modules = list(inception.children())[:-1]
        backbone = nn.Sequential(*modules)

        for layer in backbone:
            for p in layer.parameters():
                p.requires_grad = False # Freezes the backbone layers

        backbone.out_channels = 2048

        anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                           aspect_ratios=((0.5, 1.0, 2.0),))

        model = FasterRCNN(backbone, rpn_anchor_generator=anchor_generator,
                           box_predictor=FastRCNNPredictor(1024, num_classes))

        return model

In [194]:
model = make_model()

/opt/miniconda3/envs/poisson/lib/python3.7/site-packages/torchvision/models/inception.py:77: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)


# Reformat `train_one_epoch` and `evaluate` For New Model

In [213]:
!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

In [215]:
!pip install pycocotools

  Using cached Cython-0.29.21-cp37-cp37m-macosx_10_9_x86_64.whl (1.9 MB)
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-macosx_10_9_x86_64.whl size=90488 sha256=3d5863d3d1301f7eea1066a95f84b1df9912c97b06356d1e05afd3e7576ef152
  Stored in directory: /Users/richardcorrero/Library/Caches/pip/wheels/bc/cf/1b/e95c99c5f9d1648be3f500ca55e7ce55f24818b0f48336adaf
Successfully built pycocotools


In [219]:
import utils
import transforms as T
from engine import train_one_epoch, evaluate



In [224]:
ship_dir = '../dev/'
train_image_dir = os.path.join(ship_dir, 'imgs/')
valid_image_dir = os.path.join(ship_dir, 'imgs/')
masks = get_masks(ship_dir, train_image_dir, valid_image_dir)
image_names, filtered_masks = filter_masks(masks)
train_ids, train_masks, valid_ids, valid_masks = get_train_valid_dfs(
    filtered_masks
)

vessel_dataset = VesselDataset(train_masks, image_names, train_image_dir = r'../dev/imgs', mode='train')

vessel_valid_dataset = VesselDataset(valid_masks, image_names, valid_image_dir = r'../dev/imgs', mode='valid')

batch_size = 64
shuffle = True
loader = DataLoader(
            dataset=vessel_dataset,
            shuffle=shuffle,
            #num_workers = 0,
            batch_size=batch_size,
            pin_memory=torch.cuda.is_available()
        )

valid_loader = DataLoader(
            dataset=vessel_valid_dataset,
            shuffle=shuffle,
            #num_workers = 0,
            batch_size=batch_size,
            pin_memory=torch.cuda.is_available()
        )

model = make_model()

num_epochs = 30
print_freq = 100

device = torch.device('cpu')
model = model.to(device)
savepath = r'./sdad'
lr = 1e-4
weight_decay = 1e-7 # Default should be 1e-5
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

print('Starting Training...\n')
for epoch in range(num_epochs):  # loop over the dataset multiple times
    train_one_epoch(model, optimizer, loader, device, epoch, print_freq)
    print('\nEpoch %d completed. Running validation...\n' % (epoch + 1))
    evaluate(model, valid_loader, device)
    print('\nSaving Model...\n')
    torch.save(model.state_dict(), savepath)
    print('Done.\n')

Starting Training...



FileNotFoundError: [Errno 2] No such file or directory: '../dev/imgs/5fe85a9e4.jpg'

In [238]:
!python vessel_detector_test_suite.py

/opt/miniconda3/envs/poisson/lib/python3.7/site-packages/torchvision/models/inception.py:77: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
Train Size: 190620
Valid Size: 1926
Starting Training...

Traceback (most recent call last):
  File "vessel_detector_test_suite.py", line 382, in <module>
    main(savepath, backbone_loadpath)
  File "vessel_detector_test_suite.py", line 371, in main
    train_one_epoch(model, optimizer, loader, device, epoch, print_freq)
  File "/Users/richardcorrero/Projects/research/poisson/notebooks/engine.py", line 26, in train_one_epoch
    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
  File "/Users/richardcorrero/Projects/research/poisson/notebo